---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
def blight_model():
    """
    
    """
    
    # Import relevant libraries
    import pandas as pd
    import numpy as np
    import datetime as dt
    from sklearn.preprocessing import LabelEncoder
    from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import roc_auc_score, roc_curve, auc
    from sklearn.naive_bayes import GaussianNB
    from sklearn.svm import SVC
    
    # Read dataframes
    # I included the argument dtype on the training dataset to avoid a DtypeWarning on columns (11, 12, 31)
    train = pd.read_csv('readonly/train.csv', encoding='cp1252', 
                           dtype = {'zip_code': object, 'non_us_str_code': object, 'grafitti_status': object})
    test = pd.read_csv('readonly/test.csv', encoding='cp1252')
    address = pd.read_csv('readonly/addresses.csv')
    latlong = pd.read_csv('readonly/latlons.csv')
    
    # Join address and latlong in one dataframe.  Then, join address to both train and test data
    address = address.set_index('address').join(latlong.set_index('address'), how = 'left')
    train = train.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
    test = test.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
    
    # Drop NaN on target value 'compliance' --> we are just looking for compliants and or not compliants
    # Although the target variable 'compliance' is dtype('float64'). So, I will adjust it
    # to meake it categorical
    train = train.dropna(subset = ['compliance'])
    train['compliance'] = train['compliance'].astype(int)
    
    '''
    As a side note, I wanted to document my decision making regarding the attributions
    I ended up taking
    
    1.- By the moment I was selecting the variables, I wanted to add "zip_code" attribution but 
    I noticed that there were 1048 rows with wrong-formatted in the training data.  This is what I got:
    
        5.0     158831
        4.0        638
        6.0        138
        1.0        107
        3.0         60
        10.0        54
        9.0         29
        2.0         12
        7.0          7
        8.0          3
        Name: zip, dtype: int64
    
    The "zip" attribution I created, measured the lenght of each zip code
    
    '''
    
    # Drop unnecessary attributions on both train and test dataframes
    un_cols_train = []
    un_cols_test = []
    
    
    
    
    
    return # Your answer here

blight_model()

In [2]:
blight_model()

In [3]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [4]:
df_train = pd.read_csv('readonly/train.csv', encoding= 'ISO-8859-1', dtype = {'zip_code': object, 'non_us_str_code': object, 'grafitti_status': object})
df_train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [5]:
df_train.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [6]:
df_train.shape

(250306, 34)

In [7]:
df_train.describe()

,ticket_id,violation_street_number,violation_zip_code,mailing_address_str_number,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,compliance
count,250306.000000,2.503060e+05,0.0,2.467040e+05,250305.000000,250306.000000,250306.000000,250306.000000,250306.000000,250306.0,250306.000000,250306.000000,250306.000000,159880.000000
mean,152665.543099,1.064986e+04,NaN,9.149788e+03,374.423435,12.774764,6.387382,21.494506,0.125167,0.0,268.685356,48.898986,222.449058,0.072536
std,77189.882881,3.188733e+04,NaN,3.602034e+04,707.195807,9.607344,4.803672,56.464263,3.430178,0.0,626.915212,222.422425,606.394010,0.259374
min,18645.000000,0.000000e+00,NaN,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-7750.000000,0.000000
25%,86549.250000,4.739000e+03,NaN,5.440000e+02,200.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
50%,152597.500000,1.024400e+04,NaN,2.456000e+03,250.000000,20.000000,10.000000,10.000000,0.000000,0.0,140.000000,0.000000,25.000000,0.000000
75%,219888.750000,1.576000e+04,NaN,1.292725e+04,250.000000,20.000000,10.000000,25.000000,0.000000,0.0,305.000000,0.000000,305.000000,0.000000
max,366178.000000,1.415411e+07,NaN,5.111345e+06,10000.000000,20.000000,10.000000,1000.000000,350.000000,0.0,11030.000000,11075.000000,11030.000000,1.000000


In [8]:
df_test = pd.read_csv('readonly/test.csv', encoding= 'ISO-8859-1')
df_test.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [9]:
df_test.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [10]:
df_test.shape

(61001, 27)

In [11]:
df_test.describe()

,ticket_id,violation_street_number,non_us_str_code,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount
count,61001.000000,6.100100e+04,0.0,61001.000000,61001.0,61001.0,61001.000000,61001.000000,61001.000000,61001.000000
mean,331724.532811,1.256638e+04,NaN,272.714185,20.0,10.0,25.116219,0.239340,20.649711,347.895541
std,25434.932141,1.414373e+05,NaN,360.101855,0.0,0.0,36.310155,3.245894,242.375180,460.058043
min,284932.000000,-1.512600e+04,NaN,0.000000,20.0,10.0,0.000000,0.000000,0.000000,0.000000
25%,310111.000000,6.008000e+03,NaN,50.000000,20.0,10.0,5.000000,0.000000,0.000000,85.000000
50%,332251.000000,1.213400e+04,NaN,200.000000,20.0,10.0,10.000000,0.000000,0.000000,250.000000
75%,353031.000000,1.716500e+04,NaN,250.000000,20.0,10.0,25.000000,0.000000,0.000000,305.000000
max,376698.000000,2.010611e+07,NaN,10000.000000,20.0,10.0,1000.000000,250.000000,15309.000000,15558.800000


let's remember that the target variable is Compliance.  I want to know how many NaN happens in the target variable

In [12]:
mask = df_train['compliance'].isnull()
df_train[mask].shape

(90426, 34)

In [13]:
df_train.country.value_counts()


USA     250293
Cana         7
Aust         3
Egyp         2
Germ         1
Name: country, dtype: int64

In [14]:
address = pd.read_csv('readonly/addresses.csv')
address.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [15]:
address.shape

(311307, 2)

In [16]:
latlong = pd.read_csv('readonly/latlons.csv')
latlong.head()

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [17]:
latlong.shape

(121769, 3)

In [18]:
address = address.set_index('address').join(latlong.set_index('address'), how = 'left')

In [19]:
df_train.iloc[:, [11, 12, 31]].head()

,zip_code,non_us_str_code,grafitti_status
0,60606,NaN,NaN
1,48208,NaN,NaN
2,48223,NaN,NaN
3,48214,NaN,NaN
4,48206,NaN,NaN


In [20]:
df_train.iloc[:, 31].value_counts()

GRAFFITI TICKET    1
Name: grafitti_status, dtype: int64

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance'],
      dtype='object')

In [21]:
cat_lst = ['agency_name', 'inspector_name', 'violator_name', ]

In [22]:
df_train = df_train.dropna(subset = ['compliance'])
df_train['compliance'] = df_train['compliance'].astype(int)
df_train.head()


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0


In [23]:
df_train[['ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost']]

,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost
0,2004-03-16 11:40:00,2005-03-21 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0
1,2004-04-23 12:30:00,2005-05-06 13:30:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Determination,750.0,20.0,10.0,75.0,0.0,0.0
5,2004-05-01 11:50:00,2005-03-21 10:30:00,9-1-36(a),Failure of owner to obtain certificate of comp...,Responsible by Default,250.0,20.0,10.0,25.0,0.0,0.0
6,2004-06-14 14:15:00,2005-02-22 15:00:00,61-63.0500,Failed To Secure Permit For Lawful Use Of Land,Responsible by Default,750.0,20.0,10.0,75.0,0.0,0.0
7,2004-06-16 12:30:00,2005-02-22 15:00:00,61-63.0100,Noncompliance/Grant Condition/BZA/BSE,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0
8,2004-06-16 12:25:00,2005-02-22 15:00:00,61-63.0100,Noncompliance/Grant Condition/BZA/BSE,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0
9,2004-07-12 13:30:00,2005-05-31 13:30:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Default,750.0,20.0,10.0,75.0,0.0,0.0
12,2004-07-28 12:15:00,2005-02-22 15:00:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Default,750.0,20.0,10.0,75.0,0.0,0.0
13,2004-08-05 15:00:00,2005-02-22 15:00:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Default,750.0,20.0,10.0,75.0,0.0,0.0
14,2004-08-23 14:15:00,2005-02-28 15:00:00,61-63.0600,Failed To Secure Permit For Lawful Use Of Buil...,Responsible by Default,750.0,20.0,10.0,75.0,0.0,0.0


In [24]:
un_cols_train = ['agency_name', 'inspector_name', 'violator_name', 'violation_street_number', 
                 'violation_street_name','violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
                'violation_description', 'disposition']


df_train.drop(un_cols_train, axis = 1)


,ticket_id,violation_code,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,9-1-36(a),250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
1,27586,61-63.0600,750.0,20.0,10.0,75.0,0.0,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1
5,22046,9-1-36(a),250.0,20.0,10.0,25.0,0.0,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
6,18738,61-63.0500,750.0,20.0,10.0,75.0,0.0,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
7,18735,61-63.0100,100.0,20.0,10.0,10.0,0.0,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
8,18733,61-63.0100,100.0,20.0,10.0,10.0,0.0,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
9,28204,61-63.0600,750.0,20.0,10.0,75.0,0.0,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
12,18743,61-63.0600,750.0,20.0,10.0,75.0,0.0,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,IN COLLECTION,NaN,non-compliant by no payment,0
13,18741,61-63.0600,750.0,20.0,10.0,75.0,0.0,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,IN COLLECTION,NaN,non-compliant by no payment,0
14,18978,61-63.0600,750.0,20.0,10.0,75.0,0.0,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0


In [25]:
un_cols_train = ['agency_name', 'inspector_name', 'violator_name', 'violation_street_number', 
                 'violation_street_name','violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
                'violation_description', 'disposition']
un_cols_train = sorted(un_cols_train)
un_cols_train = pd.Series(un_cols_train)
un_cols_train.to_csv('un_cols_train.csv')

un_cols_train

0                    agency_name
1                           city
2                        country
3                    disposition
4                   hearing_date
5                 inspector_name
6       mailing_address_str_name
7     mailing_address_str_number
8                non_us_str_code
9                          state
10            ticket_issued_date
11         violation_description
12         violation_street_name
13       violation_street_number
14            violation_zip_code
15                 violator_name
16                      zip_code
dtype: object

In [26]:
common_cols_to_drop = ['agency_name', 'inspector_name', 'mailing_address_str_number',
                           'violator_name', 'violation_street_number', 'violation_street_name',
                           'mailing_address_str_name', 'address', 'admin_fee', 'violation_zip_code',
                           'state_fee', 'late_fee', 'ticket_issued_date', 'hearing_date', 'violation_description',
                           'fine_amount', 'clean_up_cost', 'disposition', 'grafitti_status',
                           'violation_code', 'city']
train_cols_to_drop = ['payment_status', 'payment_date', 'balance_due', 'payment_amount'] + common_cols_to_drop


train_cols_to_drop = sorted(train_cols_to_drop)
train_cols_to_drop = pd.Series(train_cols_to_drop)
train_cols_to_drop.to_csv('train_cols_to_drop.csv')
train_cols_to_drop

0                        address
1                      admin_fee
2                    agency_name
3                    balance_due
4                           city
5                  clean_up_cost
6                    disposition
7                    fine_amount
8                grafitti_status
9                   hearing_date
10                inspector_name
11                      late_fee
12      mailing_address_str_name
13    mailing_address_str_number
14                payment_amount
15                  payment_date
16                payment_status
17                     state_fee
18            ticket_issued_date
19                violation_code
20         violation_description
21         violation_street_name
22       violation_street_number
23            violation_zip_code
24                 violator_name
dtype: object

In [27]:
cocuy = ['agency_name', 'inspector_name', 'violator_name', 'non_us_str_code', 'violation_description', 
                'grafitti_status', 'state_fee', 'admin_fee', 'ticket_issued_date', 'hearing_date',
                # columns not available in test
                'payment_amount', 'balance_due', 'payment_date', 'payment_status', 
                'collection_status', 'compliance_detail', 
                # address related columns
                'violation_zip_code', 'country', 'address', 'violation_street_number',
                'violation_street_name', 'mailing_address_str_number', 'mailing_address_str_name', 
                'city', 'state', 'zip_code', 'address']
cocuy = sorted(cocuy)
cocuy = pd.Series(cocuy)
cocuy.to_csv('cocuy.csv')
cocuy

0                        address
1                        address
2                      admin_fee
3                    agency_name
4                    balance_due
5                           city
6              collection_status
7              compliance_detail
8                        country
9                grafitti_status
10                  hearing_date
11                inspector_name
12      mailing_address_str_name
13    mailing_address_str_number
14               non_us_str_code
15                payment_amount
16                  payment_date
17                payment_status
18                         state
19                     state_fee
20            ticket_issued_date
21         violation_description
22         violation_street_name
23       violation_street_number
24            violation_zip_code
25                 violator_name
26                      zip_code
dtype: object

In [28]:
original = ['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status']
original = sorted(original)
original = pd.Series(original)
original.to_csv('original.csv')


In [29]:
original_train = ['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'collection_status',
       'grafitti_status', 'compliance_detail', 'compliance']

original_train = sorted(original_train)
original_train = pd.Series(original_train)
original_train.to_csv('original_train.csv')

In [30]:
df_train.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0


In [31]:
df_test.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [32]:
df_train = df_train.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
df_train.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,IL,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,42.390729,-83.124268
27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,MI,...,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1,42.326937,-83.135118
22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,CA,...,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,42.145257,-83.208233
18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,MI,...,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,42.433466,-83.023493
18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,MI,...,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0,42.388641,-83.037858


In [33]:
df_test = df_test.set_index('ticket_id').join(address.set_index('ticket_id'), how = 'left')
df_test.head()

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.407581,-82.986642
285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,42.426239,-83.238259
285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.426239,-83.238259
285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,MI,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,42.309661,-83.122426
285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,MI,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,42.308830,-83.121116


In [34]:
df_train.shape

(159880, 35)

In [35]:
mask = pd.read_excel('model_columns.xlsx')
mask = mask['columnas']
mask = mask.tolist()
mask

['hearing_date',
 'ticket_issued_date',
 'clean_up_cost',
 'country',
 'discount_amount',
 'disposition',
 'fine_amount',
 'judgment_amount',
 'late_fee',
 'state',
 'violation_code',
 'zip_code',
 'lat',
 'lon',
 'compliance']

In [36]:
df_train_mesmo = df_train[mask]
df_train_mesmo.head()

,hearing_date,ticket_issued_date,clean_up_cost,country,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,zip_code,lat,lon,compliance
ticket_id,,,,,,,,,,,,,,,
22056,2005-03-21 10:30:00,2004-03-16 11:40:00,0.0,USA,0.0,Responsible by Default,250.0,305.0,25.0,IL,9-1-36(a),60606,42.390729,-83.124268,0
27586,2005-05-06 13:30:00,2004-04-23 12:30:00,0.0,USA,0.0,Responsible by Determination,750.0,855.0,75.0,MI,61-63.0600,48208,42.326937,-83.135118,1
22046,2005-03-21 10:30:00,2004-05-01 11:50:00,0.0,USA,0.0,Responsible by Default,250.0,305.0,25.0,CA,9-1-36(a),908041512,42.145257,-83.208233,0
18738,2005-02-22 15:00:00,2004-06-14 14:15:00,0.0,USA,0.0,Responsible by Default,750.0,855.0,75.0,MI,61-63.0500,48038,42.433466,-83.023493,0
18735,2005-02-22 15:00:00,2004-06-16 12:30:00,0.0,USA,0.0,Responsible by Default,100.0,140.0,10.0,MI,61-63.0100,48211,42.388641,-83.037858,0


In [37]:
#df_train_mesmo.loc[:,'hearing_date'] = df_train_mesmo.loc[:,'hearing_date'].astype('datetime64[ns]')
#df_train_mesmo['ticket_issued_date'] = df_train_mesmo['ticket_issued_date'].astype('datetime')

df_train_mesmo.to_csv('train_prueba.csv')

In [38]:
len(df_train_mesmo[df_train_mesmo['hearing_date'].isnull()])

227

In [39]:
len(df_train_mesmo[df_train_mesmo['ticket_issued_date'].isnull()])

0

In [40]:
len(df_train_mesmo[df_train_mesmo['lat'].isnull()])

2

In [41]:
len(df_train_mesmo[df_train_mesmo['lon'].isnull()])

2

In [42]:
df_train_mesmo['zip'] = ''

df_train_mesmo['zip'] = df_train_mesmo['zip_code'].str.len()
df_train_mesmo.tail()

C:\Users\JohnHill\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\JohnHill\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,hearing_date,ticket_issued_date,clean_up_cost,country,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,zip_code,lat,lon,compliance,zip
ticket_id,,,,,,,,,,,,,,,,
285106,2012-02-03 09:00:00,2011-12-27 14:50:00,0.0,USA,0.0,Responsible by Default,200.0,250.0,20.0,MI,22-2-88(b),48235,42.440228,-83.154829,0,5.0
284650,2012-01-09 09:00:00,2011-12-29 09:50:00,0.0,USA,0.0,Responsible by Default,1000.0,1130.0,100.0,MI,22-2-88(b),48227,42.406293,-83.171850,0,5.0
285125,2012-01-25 09:00:00,2011-12-29 09:30:00,0.0,USA,0.0,Responsible by Default,500.0,580.0,50.0,MI,22-2-88(b),48204,42.366529,-83.141897,0,5.0
284881,2012-02-07 13:30:00,2011-12-29 11:30:00,0.0,USA,0.0,Responsible by Determination,200.0,230.0,0.0,MI,22-2-61,48212,42.422081,-83.038656,1,5.0
284333,2012-01-05 09:00:00,2011-12-30 14:20:00,0.0,USA,0.0,Responsible by Default,200.0,250.0,20.0,MI,22-2-88(b),95926,42.438867,-82.959888,0,5.0


In [43]:
df_train_mesmo['zip'].value_counts()

5.0     158831
4.0        638
6.0        138
1.0        107
3.0         60
10.0        54
9.0         29
2.0         12
7.0          7
8.0          3
Name: zip, dtype: int64

In [44]:

df_train_mesmo[df_train_mesmo['zip']!=5].to_csv('que_pasa_con_zip.csv')
df_train_mesmo[df_train_mesmo['zip']!=5].head()

,hearing_date,ticket_issued_date,clean_up_cost,country,discount_amount,disposition,fine_amount,judgment_amount,late_fee,state,violation_code,zip_code,lat,lon,compliance,zip
ticket_id,,,,,,,,,,,,,,,,
22046,2005-03-21 10:30:00,2004-05-01 11:50:00,0.0,USA,0.0,Responsible by Default,250.0,305.0,25.0,CA,9-1-36(a),908041512,42.145257,-83.208233,0,9.0
19132,2005-03-01 15:00:00,2005-01-24 12:45:00,0.0,USA,0.0,Responsible by Default,250.0,305.0,25.0,MI,9-1-36(a),482227,42.373397,-83.102171,0,6.0
19400,2005-04-11 09:00:00,2005-01-26 09:15:00,0.0,USA,0.0,Responsible by Default,250.0,305.0,25.0,MI,9-1-36(a),6775,42.422083,-83.031968,0,4.0
19393,2005-04-11 09:00:00,2005-01-26 11:30:00,0.0,USA,0.0,Responsible by Determination,250.0,305.0,25.0,MI,9-1-36(a),482,42.402714,-83.161186,1,3.0
19468,2005-04-18 09:00:00,2005-01-27 11:40:00,0.0,USA,0.0,Responsible by Default,250.0,305.0,25.0,MI,9-1-36(a),4834,42.431701,-83.152368,0,4.0


In [45]:
# At this point I have made so many changes that I just wanted to confirm that I successfully
# got rid of the empty "compliance" rows. I got 159880, which is less than the original 
# 250306 original rows
df_train.shape

(159880, 35)

In [46]:
df_train_mesmo['clean_up_cost'].value_counts()

0.0    159880
Name: clean_up_cost, dtype: int64

In [47]:
df_train_mesmo['state'].value_counts()
df_train_mesmo['state'].isnull().value_counts()


False    159796
True         84
Name: state, dtype: int64

In [48]:
df_train_mesmo['discount_amount'].value_counts()

0.0      158700
25.0        605
5.0         167
10.0        155
20.0        135
50.0         43
3.0          19
30.0         17
100.0        16
350.0        15
250.0         6
40.0          1
13.0          1
Name: discount_amount, dtype: int64

In [49]:
df_train_mesmo['discount_amount'].isnull().value_counts()

False    159880
Name: discount_amount, dtype: int64

In [50]:
df_train_mesmo['disposition'].value_counts()

Responsible by Default                138340
Responsible by Admission               13701
Responsible by Determination            7644
Responsible (Fine Waived) by Deter       195
Name: disposition, dtype: int64

In [51]:
df_train_mesmo['disposition'].isnull().value_counts()

False    159880
Name: disposition, dtype: int64

In [52]:
df_train_mesmo['fine_amount'].value_counts()

250.0      86798
50.0       20415
100.0      15488
200.0      12710
500.0       6918
1000.0      4965
3500.0      3859
300.0       3768
2500.0      1545
25.0        1378
125.0        793
1500.0       264
750.0        229
0.0          195
10000.0      195
350.0        128
5000.0        85
1200.0        40
400.0         39
2000.0        16
7000.0        13
600.0         10
220.0          4
3000.0         4
1250.0         2
95.0           2
1750.0         2
150.0          2
160.0          2
450.0          1
20.0           1
655.0          1
2695.0         1
677.0          1
170.0          1
8000.0         1
270.0          1
1.0            1
970.0          1
1030.0         1
Name: fine_amount, dtype: int64

In [53]:
df_train_mesmo['fine_amount'].isnull().value_counts()

False    159880
Name: fine_amount, dtype: int64

In [54]:
df_train_mesmo['judgment_amount'].value_counts()

305.0      79621
85.0       17655
140.0      12516
250.0      11355
280.0       7177
580.0       6545
1130.0      4803
3880.0      3785
360.0       3680
130.0       2972
80.0        2760
2780.0      1506
230.0       1356
57.5        1223
167.5        723
530.0        373
1680.0       251
855.0        217
0.0          195
11030.0      195
1030.0       162
55.0         155
415.0        126
330.0         88
5530.0        85
3530.0        74
155.0         70
1350.0        40
2530.0        39
470.0         36
2230.0        14
7730.0        13
1530.0        13
780.0         12
690.0         10
3330.0         4
430.0          3
272.0          3
206.0          2
2030.0         2
134.5          2
1955.0         2
380.0          2
1405.0         2
217.0          1
1060.0         1
300.0          1
8830.0         1
1097.0         1
31.1           1
50.0           1
180.0          1
525.0          1
2994.5         1
750.5          1
774.7          1
195.0          1
Name: judgment_amount, dtype: i

In [55]:
df_train_mesmo['judgment_amount'].isnull().value_counts()

False    159880
Name: judgment_amount, dtype: int64

In [56]:
df_train_mesmo['violation_code'].value_counts()

9-1-36(a)               64414
9-1-81(a)               23145
22-2-88                 19073
9-1-104                 16927
22-2-88(b)               4879
22-2-45                  4200
9-1-105                  3619
9-1-110(a)               3147
9-1-43(a) - (Dwellin     3043
9-1-103(C)               2641
22-2-22                  2612
22-2-43                  2054
19450901                 1620
22-2-17                  1414
22-2-61                   706
22-2-83(a)(b)(c)          494
9-1-82(d) - (Dwellin      457
61-81.0100/32.0066        439
9-1-43(a) - (Structu      399
22-2-83                   347
9-1-206                   235
22-2-88(a)                235
61-5-21                   202
9-1-209                   200
22-2-21(b)                185
9-1-101                   150
9-1-107                   133
9-1-43(a) - (Stories      132
9-1-111                   130
9-1-201(a)                128
                        ...  
61-120.0100/32.0066         1
9-1-354                     1
61-86.0100

In [57]:
df_train_mesmo['late_fee'].isnull().value_counts()

False    159880
Name: late_fee, dtype: int64

In [58]:
df_train_mesmo['late_fee'].value_counts()

25.0      79621
5.0       17655
0.0       15458
10.0      12516
20.0      11354
50.0       6545
100.0      4803
350.0      3785
30.0       3680
250.0      1506
2.5        1223
12.5        723
150.0       251
75.0        217
1000.0      195
35.0        126
500.0        85
120.0        40
40.0         36
200.0        14
700.0        13
60.0         10
300.0         4
22.0          3
9.5           2
175.0         2
16.0          2
125.0         2
0.1           1
15.0          1
800.0         1
67.7          1
17.0          1
269.5         1
65.5          1
97.0          1
45.0          1
Name: late_fee, dtype: int64

In [59]:
df_train_mesmo['violation_code'].isnull().value_counts()

False    159880
Name: violation_code, dtype: int64

Once we have a clear idea about the attributions we are going to use, let's work with our dates

In [60]:

#df_train_mesmo['hearing_date']  = df_train_mesmo['hearing_date'].apply(str)

#df_train_mesmo['hearing_date'] = pd.datetime(df_train_mesmo['hearing_date'],format = "%Y-%m-%d %H:%M:%S")
#df_train_mesmo['hearing'] = df_train_mesmo.loc[:,'hearing_date'].copy()
#df_train_mesmo['hearing'] = df_train_mesmo.loc[:,'hearing_date'].apply(str)

#df_train_mesmo.loc[:,'hearing_date'] = df_train_mesmo.hearing_date.copy()
#df_train_mesmo.hearing_date.apply(type).value_counts()
df_train_mesmo.hearing_date.fillna('2019-02-01 00:00:00').apply(type).value_counts()


<class 'str'>    159880
Name: hearing_date, dtype: int64

In [61]:
from datetime import datetime
def time_gap(hearing_date_str, ticket_issued_date_str):
    if hearing_date_str.isnull(): return 73
    hearing_date = datetime.strptime(hearing_date_str, "%Y-%m-%d %H:%M:%S")
    ticket_issued_date = datetime.strptime(ticket_issued_date_str, "%Y-%m-%d %H:%M:%S")
    gap = hearing_date - ticket_issued_date
    return gap.days

In [ ]:
df_train_mesmo['hearing_date'].head()
df_train_mesmo.loc[21422, :]

In [72]:
df_train_mesmo['hearing_date'].isnull().value_counts()


False    159880
Name: hearing_date, dtype: int64

In [68]:
df_train_mesmo.hearing_date.fillna('2019-02-01 00:00:00').apply(type).value_counts()


<class 'str'>    159880
Name: hearing_date, dtype: int64

In [ ]:
encoding = c

In [ ]:
encoding='cp1252'

In [ ]:
#df_train.apply(pd.value_counts) # use it only with columns that could be aggregated